## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

import numpy as np

In [0]:

#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [2]:
from google.colab import drive
#drive.mount('/gdrive',force_remount=True)

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import os

In [4]:
#data = pd.read_csv('/drive//sample_data')
print(os.getcwd())
!ls

/content
drive  sample_data


In [0]:
data = pd.read_csv("/content/drive/My Drive/internalLab/prices.csv", delimiter=",")

### Check all columns in the dataset

In [6]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data['volume']=data.iloc[0:1001,4]/1000000

In [10]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [0]:
data_new=data.iloc[0:1000,:]

In [12]:
data_new.shape

(1000, 5)

In [13]:
data_new.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = data_new.iloc[:,0:4]
Y = data_new.iloc[:,4]
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')





### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)


In [18]:
train_y.reshape(1, -1)

array([[7.151000e-01, 4.550800e+00, 7.499000e-01, 2.465900e+00,
        5.147000e-01, 3.293000e-01, 4.277900e+00, 2.856000e+00,
        4.545400e+00, 1.426600e+00, 2.521200e+01, 3.252400e+00,
        1.964100e+00, 3.959400e+00, 9.246000e-01, 2.489500e+00,
        3.624100e+00, 1.197120e+01, 1.637000e+01, 7.449000e-01,
        4.564300e+00, 1.731300e+01, 2.652000e+00, 2.849400e+00,
        3.945000e-01, 5.421900e+00, 1.975600e+00, 7.165000e-01,
        1.559430e+01, 1.120200e+00, 3.227300e+00, 1.793200e+00,
        8.385500e+00, 2.971500e+00, 1.526800e+00, 6.109400e+00,
        8.980000e-01, 5.985370e+01, 3.450400e+00, 9.280400e+00,
        5.156400e+00, 4.406600e+00, 3.637000e+00, 1.483400e+00,
        5.405000e-01, 6.819000e-01, 1.197240e+01, 1.541000e+00,
        7.125000e-01, 1.066340e+01, 1.653910e+01, 1.357340e+01,
        5.341000e-01, 6.686170e+01, 2.946600e+00, 5.875500e+00,
        1.524500e+00, 7.389200e+00, 8.847000e-01, 2.033000e+00,
        2.701000e+00, 2.206600e+00, 1.05

In [0]:
train_x.shape

(700, 4)

In [19]:
test_y.shape

(300,)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w1 = tf.random_normal(shape=(4,5))
b1 = tf.zeros(shape=(5))

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random_normal(shape=(5,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out
  


3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        current_loss =loss(net2,train_y)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2,current_loss

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [28]:
      for i in range(100):
        w1, b1,w2,b2,current_loss = train(train_x, train_y, w1, b1,w2,b2)
        print("Loss at step {:d}: {:.3f}".format(i, current_loss))      #Training loss at each iteration

        #can you please comment on  this I am not sure why am getting this error(regarding the string format). The classroom faculty also couldn't help with this.

ValueError: ignored

In [29]:
for i in range(100):
        w1, b1,w2,b2,current_loss = train(train_x, train_y, w1, b1,w2,b2)
        print("Loss at step {:d}: ".format(i, current_loss))   

Loss at step 0: 
Loss at step 1: 
Loss at step 2: 
Loss at step 3: 
Loss at step 4: 
Loss at step 5: 
Loss at step 6: 
Loss at step 7: 
Loss at step 8: 
Loss at step 9: 
Loss at step 10: 
Loss at step 11: 
Loss at step 12: 
Loss at step 13: 
Loss at step 14: 
Loss at step 15: 
Loss at step 16: 
Loss at step 17: 
Loss at step 18: 
Loss at step 19: 
Loss at step 20: 
Loss at step 21: 
Loss at step 22: 
Loss at step 23: 
Loss at step 24: 
Loss at step 25: 
Loss at step 26: 
Loss at step 27: 
Loss at step 28: 
Loss at step 29: 
Loss at step 30: 
Loss at step 31: 
Loss at step 32: 
Loss at step 33: 
Loss at step 34: 
Loss at step 35: 
Loss at step 36: 
Loss at step 37: 
Loss at step 38: 
Loss at step 39: 
Loss at step 40: 
Loss at step 41: 
Loss at step 42: 
Loss at step 43: 
Loss at step 44: 
Loss at step 45: 
Loss at step 46: 
Loss at step 47: 
Loss at step 48: 
Loss at step 49: 
Loss at step 50: 
Loss at step 51: 
Loss at step 52: 
Loss at step 53: 
Loss at step 54: 
Loss at step 55: 
Lo

### Get the shapes and values of W and b

In [30]:
print(w1)

Tensor("sub_513:0", shape=(4, 5), dtype=float32)


In [32]:
print(w2)

Tensor("sub_511:0", shape=(5, 1), dtype=float32)


In [34]:
print(b1)

Tensor("sub_514:0", shape=(5,), dtype=float32)


In [35]:
print(b2)

Tensor("sub_512:0", shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [38]:
y_pred,_=prediction(test_x,w1,b1,w2,b2)
y_pred


<tf.Tensor 'add_211:0' shape=(300, 1) dtype=float32>

In [42]:
test_y - y_pred


<tf.Tensor 'sub_517:0' shape=(300, 300) dtype=float32>

In [0]:
from sklearn import metrics

In [45]:
cm=metrics.r2_score(test_y,y_pred)
print(cm)

TypeError: ignored

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf


### Load the given Iris data using pandas (Iris.csv)

In [0]:
data2 = pd.read_csv("/content/drive/My Drive/internalLab/iris.csv", delimiter=",")

In [0]:
data2.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
dummies = data2["Species"].str.get_dummies(" ") 


In [0]:
dummies.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


### Splitting the data into feature set and target set

In [0]:
X_i = data2.iloc[:,0:5]
Y_i = dummies
train_x,test_x,train_y,test_y=train_test_split(X_i,Y_i,test_size=.30,random_state=2)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()

# Add Dense Layer which provides 10 Outputs after applying softmax
#model.add(tf.keras.layers.Dense(3,input=4, activation='softmax'))

model.add(tf.keras.layers.Dense(3, input_shape=(5,),activation='softmax'))
# Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


### Model Training 

In [0]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50,
          batch_size = train_x.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 4ms/sample - loss: 1.1557 - acc: 0.3619 - val_loss: 12.4912 - val_acc: 0.2889
Epoch 2/50
105/105 [==============================] - 0s 65us/sample - loss: 10.6832 - acc: 0.3524 - val_loss: 10.9826 - val_acc: 0.3333
Epoch 3/50
105/105 [==============================] - 0s 31us/sample - loss: 12.1819 - acc: 0.3333 - val_loss: 22.8707 - val_acc: 0.2889
Epoch 4/50
105/105 [==============================] - 0s 40us/sample - loss: 19.9528 - acc: 0.3524 - val_loss: 7.7348 - val_acc: 0.2889
Epoch 5/50
105/105 [==============================] - 0s 38us/sample - loss: 7.0968 - acc: 0.2762 - val_loss: 19.1238 - val_acc: 0.2889
Epoch 6/50
105/105 [==============================] - 0s 44us/sample - loss: 16.3483 - acc: 0.3524 - val_loss: 16.0846 - val_acc: 0.3333
Epoch 7/50
105/105 [==============================] - 0s 36us/sample - loss: 16.2678 - acc: 0.3333 - val_loss: 29.4788 - val_acc: 0.2889


### Model Prediction

In [0]:
model.predict(test_x)

array([[2.00499490e-01, 3.81500155e-01, 4.18000370e-01],
       [4.07079279e-01, 5.34139514e-01, 5.87812141e-02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [6.19137252e-04, 2.04135827e-03, 9.97339547e-01],
       [2.06249062e-09, 9.93805287e-08, 9.99999881e-01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [7.72290176e-10, 1.96811101e-08, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [6.55504963e-19, 2.50469442e-16, 1.00000000e+00],
       [7.83044323e-20, 4.48211256e-17, 1.00000000e+00],
       [1.37002624e-17, 4.18137984e-15, 1.00000000e+00],
       [4.00301333e-14, 2.35428517e-12, 1.00000000e+00],
       [3.37551743e-01, 5.71200252e-01, 9.12479907e-02],
       [1.62055011e-37, 1.61553775e-30, 1.00000000e+00],
       [1.50679438e-23, 1.14455947e-18, 1.00000000e+00],
       [1.76572372e-17, 3.28670086e-15, 1.00000000e+00],
       [0.00000000e+00, 2.51135

### Save the Model

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?